# 実践問題

## ライブラリのインポート

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set()

## データの読み込み

In [ ]:
# データを読み込みます
raw_data = pd.read_csv('1.04. Real-life example.csv')

# 頭の5つのデータを表示します
raw_data.head()

## 前処理

### 変数の記述統計量の確認

In [ ]:
# カテゴリ変数も表示するために、include='all'の引数を指定します
raw_data.describe(include='all')

### 変数の選択

In [ ]:
# Modelの列を削除します
data = raw_data.drop(['Model'],axis=1)

# 改めて記述統計量を確認します
data.describe(include='all')

### 欠損値の扱い

In [ ]:
# data.isnull() を使ってNullデータを確認します 
# Trueの場合、データのが無いという事を意味していますので、その値を足していってNullの数を数えます
data.isnull().sum()

In [ ]:
# 全ての欠損値を削除します
data_no_mv = data.dropna(axis=0)

In [ ]:
# 改めて記述統計量を確認します
data_no_mv.describe(include='all')

### 確率密度関数

In [ ]:
# 確率密度関数を表示します
sns.distplot(data_no_mv['Price'])

### 外れ値の処理

In [ ]:
# 1％の外れ値を取り除いた変数を作成します
q = data_no_mv['Price'].quantile(0.99)
data_1 = data_no_mv[data_no_mv['Price']<q]
data_1.describe(include='all')

In [ ]:
# 改めて確率密度関数を表示します
sns.distplot(data_1['Price'])

In [ ]:
# 他の量的変数についても同じように処理していきます
sns.distplot(data_no_mv['Mileage'])

In [ ]:
q = data_1['Mileage'].quantile(0.99)
data_2 = data_1[data_1['Mileage']<q]

In [ ]:
sns.distplot(data_2['Mileage'])

In [ ]:
sns.distplot(data_no_mv['EngineV'])

In [ ]:
# 車の排気量は最大でも6.5が一般的なので、その値のデータを抽出します
data_3 = data_2[data_2['EngineV']<6.5]

In [ ]:
sns.distplot(data_3['EngineV'])

In [ ]:
sns.distplot(data_no_mv['Year'])

In [ ]:
# 古い年のデータを取り除いていきます
q = data_3['Year'].quantile(0.01)
data_4 = data_3[data_3['Year']>q]

In [ ]:
sns.distplot(data_4['Year'])

In [ ]:
# 削除した列（インデックス）があるので、再度インデックスを作成していきます
data_cleaned = data_4.reset_index(drop=True)

In [ ]:
# 記述統計量を表示します
data_cleaned.describe(include='all')

## 最小二乗法の前提の確認

In [ ]:
# 散布図の作成
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize =(15,3)) 
ax1.scatter(data_cleaned['Year'],data_cleaned['Price'])
ax1.set_title('Price and Year')
ax2.scatter(data_cleaned['EngineV'],data_cleaned['Price'])
ax2.set_title('Price and EngineV')
ax3.scatter(data_cleaned['Mileage'],data_cleaned['Price'])
ax3.set_title('Price and Mileage')


plt.show()

In [ ]:
# 価格が指数のように分布していることから、対数変換をしていきます
sns.distplot(data_cleaned['Price'])

### 前提の緩和

In [ ]:
# Priceを対数に変換していきます
log_price = np.log(data_cleaned['Price'])

# データフレームに追加します
data_cleaned['log_price'] = log_price
data_cleaned

In [ ]:
# 改めて散布図を作成します
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize =(15,3))
ax1.scatter(data_cleaned['Year'],data_cleaned['log_price'])
ax1.set_title('Log Price and Year')
ax2.scatter(data_cleaned['EngineV'],data_cleaned['log_price'])
ax2.set_title('Log Price and EngineV')
ax3.scatter(data_cleaned['Mileage'],data_cleaned['log_price'])
ax3.set_title('Log Price and Mileage')


plt.show()


In [ ]:
# 値段の対数のデータを使っていますので、元の値段の列を削除します
data_cleaned = data_cleaned.drop(['Price'],axis=1)

### 多重共線性

In [ ]:
# データフレームの列を見ていきましょう
data_cleaned.columns.values

In [ ]:
# 多重共線性を確認するためのモジュールをインポートしていきます
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 多重共線性を確認するための列を指定した変数を作成します
variables = data_cleaned[['Mileage','Year','EngineV']]

# 新しいデータフレームを作成します
vif = pd.DataFrame()

# それぞれのVIFの値を求めていきます
vif["VIF"] = [variance_inflation_factor(variables.values, i) for i in range(variables.shape[1])]
# 対応する列の名前を追加します
vif["Features"] = variables.columns

In [ ]:
# 結果の確認
vif

In [ ]:
# Yearに関するデータを削除します
data_no_multicollinearity = data_cleaned.drop(['Year'],axis=1)

## ダミー変数の作成

In [ ]:
# get_dummiesメソッドを使ってダミー変数を作成します
data_with_dummies = pd.get_dummies(data_no_multicollinearity, drop_first=True)

In [ ]:
# 結果を表示します
data_with_dummies.head()

### データの並べ替え等

In [ ]:
# 全ての列の名前を表示します
data_with_dummies.columns.values

In [ ]:
# 従属変数、独立変数、ダミー変数で並べかをしていきます
cols = ['log_price', 'Mileage', 'EngineV', 'Brand_BMW',
       'Brand_Mercedes-Benz', 'Brand_Mitsubishi', 'Brand_Renault',
       'Brand_Toyota', 'Brand_Volkswagen', 'Body_hatch', 'Body_other',
       'Body_sedan', 'Body_vagon', 'Body_van', 'Engine Type_Gas',
       'Engine Type_Other', 'Engine Type_Petrol', 'Registration_yes']

In [ ]:
# 新しいデータフレームに並べ替えたデータを入れていきます
data_preprocessed = data_with_dummies[cols]
data_preprocessed.head()

## 線形回帰モデル

### 入力とターゲットの宣言

In [ ]:
# ターゲットは'log price'となります
targets = data_preprocessed['log_price']

# 入力は従属変数以外となります
inputs = data_preprocessed.drop(['log_price'],axis=1)

### データの標準化

In [ ]:
# 標準化のモジュールをインポートします
from sklearn.preprocessing import StandardScaler

# オブジェクトの作成
scaler = StandardScaler()
# モデルにデータをフィットさせます
scaler.fit(inputs)

In [ ]:
# 標準化した値を変数に入れます
inputs_scaled = scaler.transform(inputs)

### 訓練データとテストデータの分割

In [ ]:
# 分割するためのモジュールのインポート
from sklearn.model_selection import train_test_split

# random_stateを365としてデータを訓練80、テスト20で分割します
x_train, x_test, y_train, y_test = train_test_split(inputs_scaled, targets, test_size=0.2, random_state=365)

### 回帰の作成

In [ ]:
# オブジェクトの作成
reg = LinearRegression()
# モデルにデータをフィットさせます
reg.fit(x_train,y_train)

In [ ]:
# 出力を表示します
y_hat = reg.predict(x_train)

In [ ]:
# 出力とターゲットを散布図にして表示します
plt.scatter(y_train, y_hat)
# 軸に名前を付けます
plt.xlabel('Targets (y_train)',size=18)
plt.ylabel('Predictions (y_hat)',size=18)
# 軸の範囲を決めます
plt.xlim(6,13)
plt.ylim(6,13)
plt.show()

In [ ]:
# 残差を図示します
sns.distplot(y_train - y_hat)

# タイトルを入れます
plt.title("Residuals PDF", size=18)

In [ ]:
# 決定係数を求めます
reg.score(x_train,y_train)


### 重みとバイアス

In [ ]:
# バイアスを求めます
reg.intercept_

In [ ]:
# 重みを求めます
reg.coef_

In [ ]:
# まとめ表を表示します
reg_summary = pd.DataFrame(inputs.columns.values, columns=['Features'])
reg_summary['Weights'] = reg.coef_
reg_summary

In [ ]:
# ブランドの種類を確認します
data_cleaned['Brand'].unique()

## テスト

In [ ]:
# 作成したモデルにテストデータを入れて出力を求めます
y_hat_test = reg.predict(x_test)

In [ ]:
# ターゲットと出力の散布図を作成します
plt.scatter(y_test, y_hat_test, alpha=0.2)
plt.xlabel('Targets (y_test)',size=18)
plt.ylabel('Predictions (y_hat_test)',size=18)
plt.xlim(6,13)
plt.ylim(6,13)
plt.show()

In [ ]:
# 予測された値を数値として確認します
df_pf = pd.DataFrame(np.exp(y_hat_test), columns=['Prediction'])
df_pf.head()

In [ ]:
# データフレームに予測した値を入れていきます
df_pf['Target'] = np.exp(y_test)
df_pf

In [ ]:
# 上記では古いインデックスが残っているので、それを下のコードで削除していきます
# The code was: data_cleaned = data_4.reset_index(drop=True)

# インデックスをリセットしていきます
y_test = y_test.reset_index(drop=True)

# 結果を確認します
y_test.head()

In [ ]:
# ターゲットの値を追加していきます
df_pf['Target'] = np.exp(y_test)
df_pf

In [ ]:
# 残差の列を追加します
df_pf['Residual'] = df_pf['Target'] - df_pf['Prediction']

In [ ]:
# 差の割合も計算し、列に追加します
df_pf['Difference%'] = np.absolute(df_pf['Residual']/df_pf['Target']*100)
df_pf

In [ ]:
# 統計量を表示します
df_pf.describe()

In [ ]:
# 以下のコードで全ての列が表示されるようにしていきます
pd.options.display.max_rows = 999
# 結果を小数点以下2桁にします 
pd.set_option('display.float_format', lambda x: '%.2f' % x)
# 差の割合を小さい順に並べ替えていきます
df_pf.sort_values(by=['Difference%'])